#  Combining the Components into the Full Pipeline
* The the code from the various notebooks are combined into a single notebook  

# Lane Class
* The Lane class holds information about the right and left lane boundaries. 

## Video Processing Pipeline.  
* Run the code below to generate output movies
Movies will be stored in ./polyfits under the same name as the source movie

In [1]:
import numpy as np
import glob
import edge_utils as eu

import pickle

from moviepy.editor import *
import cv2


def computeEdges(binary):   
    '''
    Three edge detection methods based on Sobel were used.
    The results of each were accumlated in the numpy array summer
    Thus, in this case summer could have a max of 3 for an
    edge pixel that was detected in all three methods.  Thresholds
    were determined by trial and error
    ''' 
    summer = eu.dir_threshold(binary, 3, thresh=(.7,1.4)) 
    summer = np.add(summer, eu.abs_sobel_thresh(binary, 'x', .2,1.2))  
    summer = np.add(summer, eu.abs_sobel_thresh(binary, 'y', .4 ,1.1)) 

   # dilate = cv2.dilate(summer,dilation_kernel,iterations = 3) 
    print("max at end of edge summation {}".format(np.max(summer)))
    return summer

def measure_curvature_meters(ypoints, xfit):
     
    '''
    Define y-values for the radius of curvature
    Use a maximum y-value, corresponding to the bottom of the image
    Note: this was taken from the instructional material
    '''
    
    y_eval = np.max(ypoints)

    curverad = ((1 + (2*xfit[0]*y_eval + 
                           xfit[1])**2)**1.5) / np.absolute(2*xfit[0])
    ym_pix = 30/270
     
    return curverad*ym_pix

def computeEMD(object1, object2):
    '''
    This function returns the cross entropy and earth
    movers distance between tho histograms.  The histograms
    are retrieved from the Lane object. Lane is shown below.
    '''
    #Lane.returnLane()[0] returns the histogram of counts
    #along the columns of the image.  We retrieve two
    #histograms for calling Shannon cross entropy
    lane1_histo = object1.returnLane()[0]
    lane2_histo = object2.returnLane()[0]
    
    #important to turn histograms into proper statistical distributions
    #divide by total number of activated pixels
    lane1_histo /= np.sum(lane1_histo)
    lane2_histo /= np.sum(lane2_histo)
    
    print("Lane 1 entropy {}".
          format(stats.entropy(lane1_histo)))
    print("Lane 2 entropy {}".
          format(stats.entropy(lane2_histo)))
    print("Lane 1, Lane 2 cross entropy {}".
          format(stats.entropy(lane1_histo,lane2_histo)))

    #I was not able to use the wassertstein distance (aka EMD)
    #on the platform.  It worked on my home computer
    #print("Lane 1, Lnae 2 Earth Mover Distance {}".
    #      format(stats.wasserstein_distance(lane1_histo,lane2_histo)))
    
    ##for the future detemine a rule to stop further sampling
    continue_processing = True
    
    return continue_processing

def computeFits(samples, side=0):
  '''
  Computes the line fits for a series of samples.  
  '''
  offset = 0 if side==0 else 640
  fit = []
  #loop over line data and compute polylines
  for indx,sample in enumerate(samples):
    #print(arg)
    
    if (len(sample)>0):
      
      y = [row[0] for row in sample]
      x = [row[1] for row in sample]
   
      #format x and y.  x should be reversed 
      #xback = x[::-1] 
      xfit = np.polyfit(y,x, deg=2)       
 
      #print("indx {}".format(indx))
      #print(xfit)
      fit.append(xfit)
    
  return fit 
 

In [8]:
def renderCenterLine(image):
    
    if len(center_fits[0])>0:
 
          ave_xfit = (np.nanmean([v[0] for v in center_fits]),
                      np.nanmean([v[1] for v in center_fits]),
                      np.nanmean([v[2] for v in center_fits]))
 
          curverad = measure_curvature_meters(yp, ave_xfit)      
 
          z = np.poly1d(ave_xfit) 
          ##use the meters/pixel calculation
          xm_pix = 3.7/700
          fitted_points = np.array([[320+int(z(y)),int(y)] for y in yp])
          distance = xm_pix * (320-np.mean(z(ave_xfit)))
          cv2.polylines(image, [fitted_points], False, (255,0,255), 4)
        
          return distance

In [9]:
 def renderLineFits(image, side):
           
  #xfits is a list if lists, one list for each line (FWHM-,FWHM+, etc)     
  offset = 0 if side==0 else 640
  cvals = [(0,255,0), (0,0,255), (0,255,255), (255,255,0)]
  plotloc = 100  #y axis for text output 

    
  fits = left_xfits if side==0 else right_xfits
  curverad = 0
  for indx, fit in enumerate(fits):
       if len(fit)>0:
 
          ave_xfit = (np.nanmean([v[0] for v in fit]),
                      np.nanmean([v[1] for v in fit]),
                      np.nanmean([v[2] for v in fit]))
 
          curverad += measure_curvature_meters(yp, ave_xfit)      
 
          z = np.poly1d(ave_xfit)  
 
          fitted_points = np.array([[offset+int(z(y)),int(y)] for y in yp])

          cv2.polylines(image, [fitted_points], False, cvals[indx], 4)
         # cv2.putText(image, "radius in pix: {}".format(round(curverad,2)), 
         #            (500, plotloc), cv2.FONT_HERSHEY_COMPLEX_SMALL, 
         #            1.0, cvals[indx])
    
    #plot the data points along side the polylines
    #for i, pt in enumerate(x):
      # print(x[pt])
    #  cv2.rectangle(image, 
    #                (x[i],y[i]), (x[i]+20,y[i]+20), 
    #                cvals[indx], -1)  
          plotloc += 50 #move the plot down
  return np.mean(curverad)

def computeXFits(indices):
    '''
    Simple function to fit lines from points
    '''
    y = [row[0] for row in indices]
    x = [row[1] for row in indices]
    
    #reverse x for polyfit
    xback = x[::-1] 
    xfit = np.polyfit(y,x, deg=2)      
    return xfit

def update_overall_mask(mask, inds, side=0):
  '''
  The overall mask is used see how many pixels fall in the 
  current lane.  If a solid majority are in the mask, we
  skip the re sampling and polyline step
  '''
  #10 y-points is sufficient for masking
  ypsmall = np.linspace(1, 720, 10) 
  #double the y values to make convex hull
  yps = np.append(ypsmall,ypsmall)
  #reverse yps to make clockwise side of convex hull
  ypsrev = yps[::-1]    

  start_xfit = computeXFits(inds[0])
  stop_xfit = computeXFits(inds[1]) 

  #start is for the left side of the mask
  #stop is for the right side of the mask
  #prepare the points for the convex hull
  zstart = np.poly1d(start_xfit)
  zstop = np.poly1d(stop_xfit)

  fitted_starts = np.array([[int(zstart(y)),int(y)] for y in ypsmall])
  fitted_stops = np.array([[int(zstop(y)),int(y)] for y in ypsmall[::-1]])
  points = np.append(fitted_starts, fitted_stops, axis=0)
  #for the offset
  start_pt = left[0] if side==0 else right[0]
  stop_pt = left[1] if side==0 else right[1]
  
  ##create the convex hull                                                                    
  filler = cv2.convexHull(np.array(points), True, True)
  #fill up the convex hull to create a mask
  mask = cv2.drawContours(overall_mask[:,start_pt:stop_pt], [filler],
                                  -1, (255 , 255 , 255),thickness=cv2.FILLED)     


def returnLineIndices(binary_warped, side=0, height=720):
    
    '''
    This function creates a set of polylines by splitting the 
    image into stripes (e.g., 0:100 across the right and
    left side of the image).  Todo:  stop processing when
    the line appears good.  
    '''
    
    #the indices and the four key line componts FWHM-, peak, FWHM+ and mean
    #side is setup to determine left (0) or right (1)
    start_inds,stop_inds,peak_inds,mean_inds = [],[],[],[]
    start_pt = left[0] if side==0 else right[0]
    stop_pt = left[1] if side==0 else right[1]
          
    #offset keeps track of left (offset 0) or right (offset 640)
    #for now its hardcoded but should be made more general
    offset = 0 if side==0 else 640
    
    #the global_lane covers the entire height of the image on the right and left
    global_lane = Lane(binary_warped, (start_pt, stop_pt ), (0,height), 
                          offset = offset, default_value=400)
    
    #this is the fall back number in case we start from nothing.
    lane_default = global_lane.returnLane()[3]    
    
    #windows march through the stripes of the image.  2 is for top botton. 
    #4 is for quarters.  Note that 2,4 would yield 6 unique stripes
    
    windows = [2,3,4]
 
    for window in windows:  
      for strip in range(window):
         winsize = height//window
         strip_start = strip*winsize
         strip_stop = (strip*winsize) + winsize

         strip_midpt = (strip_start+strip_stop)//2

         #create Lane object to hold points and line sample data
         lane = Lane(binary_warped, (start_pt, stop_pt), (strip_start, strip_stop), 
                           offset=offset, default_value=lane_default)
       
         histo, start, stop, peak_indx  = lane.returnLane()
       
         #accumulate line points 
         stop_inds.append([strip_midpt,stop])
         start_inds.append([strip_midpt,start])
         stop_inds.append([strip_midpt,stop]) 
         peak_inds.append([strip_midpt,peak_indx]) 
         mean_inds.append([strip_midpt,start//3+stop//3+peak_indx//3])

    return start_inds, stop_inds, peak_inds, mean_inds

# Process Image Method
This method runs once per frame and contains 

In [16]:
    
def process_image(image):

  '''
  This is the iterator function that processes images thru moviepy
  The function compares the edge image to a mask based on the 
  convex hull of the most recently accepted left and right lines.
  If the percentage of active pixels is high (mean good match),
  then a new poly line is not calculated
  '''
  
  #image = np.copy(img)
  global left_poly_update_status, right_poly_update_status
  global left_start_inds, left_stop_inds 
  global right_start_inds, right_stop_inds
  global left_peak_inds, right_peak_inds
  global left_mean_inds, right_mean_inds
  global left_no_pix, right_no_pix

  blank_canvas = np.zeros_like(image)
    
  #correct the camera for lens distortion
  dst = cv2.undistort(image, mtx, dist, None, mtx)

  s = computeEdges(eu.color_mask(dst)) 

  HLS = cv2.cvtColor(dst, cv2.COLOR_BGR2HLS)
  #s = np.add(s,computeEdges(HLS[:,:,0]))

  #s = np.add(s, computeEdges(HLS[:,:,1]))  

  s = np.add(s, computeEdges(HLS[:,:,2]))  
  XYZ = cv2.cvtColor(dst, cv2.COLOR_BGR2XYZ)
  s = np.add(s, computeEdges(XYZ[:,:,0]))
  
  #the uniques (ranging 0:5 typically) are thresholded
  summer_max = np.max(np.unique(s))
  #print("summer max {}".format(summer_max))
  
  #less than 10% pixels provide good signal
  #for level in np.unique(s):
  #percent_active = np.count_nonzero(s>level, axis=None)/np.size(s)
  #print("level {}; %pix {}".format(level,percent_active))

  #less than 10% pixels provide good signal
  #for level in np.unique(s):
  #  percent_active = np.count_nonzero(s>level, axis=None)/np.size(s)
  #  print("level {}; %pix {}".format(level,percent_active))
  #  if percent_active < .10:
  #      break
         
  #if percent_active < .10:
  #      break
  level = 2   
  print("threshold {}".format(level))
  s[s<=level] = 0
  s[s>level] = 1 

  h,w = np.shape(s)
  
  print("percent activated pixels {}".
       format(np.count_nonzero(s, axis=None)/np.size(s)))

  morphed = cv2.dilate(s,kernel,iterations = 1) 
  #print("max of morphed is {}".format(np.max(morphed)))

  binary_warped = cv2.warpPerspective(morphed, M, (w,h)) 
    
  #image_warped holds the graphics on a birds eye view
  #once graphics are drawn, it will be transformed back
  blank_warped = cv2.warpPerspective(blank_canvas, M, (w,h)) 
  #print(np.max(binary_warped))

  #decide whether to create a new polyline here if low overlap
  #or low of pixels

  #compare the right size to percent overlap and minimum pixel counts, 
  noverlapping = np.sum(cv2.bitwise_and(overall_mask[:,:w//2], 
                                       binary_warped[:,:w//2]))
    
  percent_overlapping = noverlapping/np.sum(binary_warped[:,:w//2])
  print("left overlapping pixels {}; percent of total {}".
        format(noverlapping, percent_overlapping))

  if (percent_overlapping < 0.9) or (100 < noverlapping < 400):
        left_poly_update_status = True
        print("left_poly")
        left_no_pix = 0
  elif noverlapping < 50 and left_no_pix <30:
        left_no_pix +=1
        overall_mask[:,:640] = 0
        left_poly_update_status = False
  #else:
  #      left_poly_update_status = True
        
        
  #compare the right size to percent overlap and minimum pixel counts, 
  noverlapping = np.sum(cv2.bitwise_and(overall_mask[:,w//2:], 
                                       binary_warped[:,w//2:]))
    
  percent_overlapping = noverlapping/np.sum(binary_warped[:,w//2:])
  print("right overlapping pixels {}; percent of total {}".
        format(noverlapping, percent_overlapping))
    
  if (percent_overlapping < 0.9) or (100 < noverlapping < 400):
        right_poly_update_status = True
        print("right_poly")
        right_no_pix = 0
  elif noverlapping < 50 and right_no_pix <30:
        right_no_pix += 1
        overall_mask[:,640:] = 0
        right_poly_update_status = False
  #else:
  #      right_poly_update_status = True
        
  #left_poly_update_status = True
  #right_poly_update_status = True 
    
  if left_poly_update_status:
       #left_start_inds,left_stop_inds,left_peak_inds,left_mean_inds = returnLineIndices(binary_warped, side=0, height=h)
       l = returnLineIndices(binary_warped, side=0, height=h)
       lfits = computeFits(l,side=0)    
       update_overall_mask(overall_mask, l, side=0)
       for i in range(4):
           if len(left_xfits[i])==20:
            del left_xfits[i][0]
           left_xfits[i].append(lfits[i])   
       print("****update left mask run")
    
  if right_poly_update_status:
       #left_start_inds,left_stop_inds,left_peak_inds,left_mean_inds = returnLineIndices(binary_warped, side=0, height=h)
       r = returnLineIndices(binary_warped, side=1, height=h)
       rfits = computeFits(r, side=1)     
       update_overall_mask(overall_mask, r, side=1)  
       for i in range(4):
           if len(right_xfits[i])==20:
              del right_xfits[i][0]
           right_xfits[i].append(rfits[i])
       print("****update right mask run")
    
  if left_poly_update_status and right_poly_update_status:   
        ##update center
        c = [(l+r)/2 for l,r in zip(lfits[3],rfits[3])]
        print(c)
        #center = np.mean(c)
        #print("CENTER {}".format(center))
        if len(center_fits)==20:
              del center_fits[0]
        center_fits.append(c)
            
          
  #use outimage to see birds eye edge image with lines
  #outimage = np.dstack((255*binary_warped, 255*binary_warped, 255*binary_warped))
  #renderLineFits(outimage, side=0) 
  #renderLineFits(outimage, side=1)

 
  curve_left = renderLineFits(blank_warped, side=0) 
  curve_right = renderLineFits(blank_warped, side=1)
  mean_curve = np.mean([curve_left, curve_right])
 
  #offcenter is the offset from the center of lane
  offcenter = renderCenterLine(blank_warped)
 
    
 # outimage = np.dstack((255*binary_warped,255*binary_warped,255*binary_warped))
       #img_warp
  dewarped = cv2.warpPerspective(blank_warped, Mt, (1280,720)) 
  if mean_curve:
        if len(curvatures)==20:
            del curvatures[0]
        curvatures.append(mean_curve)
        
        cv2.putText(dewarped, "curve radius in (m): {}".
                    format(round(np.mean(curvatures),2)),               
                          (500, 200), cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                        2.0, (255,255,255))
  if offcenter:
        if len(offsets)==20:
            del offsets[0]
        offsets.append(offcenter)
     
        offdir = 'LEFT' if offcenter<=0 else 'RIGHT'
        cv2.putText(dewarped, offdir + " of center (m): {}".
                              format(round(np.mean(offsets),2)),               
                             (500, 300), cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                             2.0, (255,255,255))
  
  bigout = cv2.addWeighted(dewarped, 1.0, dst, .5, .5)
  return bigout

In [17]:
test = []
if not test:
    print("shoot")

shoot


In [18]:

from scipy.signal import argrelextrema
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
#export PYTHONPATH=/usr/local/lib/python2.7/site-packages:$PYTHONPATH
from scipy.signal import savgol_filter

'''
The Lane class encapulates line/lane properties and includes
logic to find histogram peaks and their Full Width and Half Max
values to use in segmenting the lane
'''

class Lane:
    
    smoothed = []
    imclip = []
    lane = []
 
    
    def __init__(self, binary, xcoords, ycoords, offset, default_value):
      
      self.binary = binary
      self.xcoords = xcoords
      self.ycoords = ycoords
      self.default_value = default_value
      self.offset = offset
        
      #clip the image to between the x and y coordinates supplied
      #these usually correpond to the image stripe and left/right
      self.imclip = binary[ycoords[0]:ycoords[1], xcoords[0]:xcoords[1]]
      histogram = np.sum(self.imclip, axis=0)
      h,w = np.shape(self.imclip)
    
      #use the sav_gol filter to smooth the peaks
      #prior to peak finding
      self.smoothed = savgol_filter(histogram,65,1)
 
      self.lane = self.makeLane( self.smoothed)
 
      
    def makeLane(self, histo):
      '''
      This funcitons determines the peaks and half max
      for the supplied histogram
      '''
      
      peak_indx = np.argmax(histo)
      peak_half_max = histo[peak_indx]/2
   
      starts = np.where(histo[:peak_indx]<peak_half_max)[0]
      stops = np.where(histo[peak_indx:]<peak_half_max)[0]
    
      if np.size(peak_indx)==0:
        peak_indx = self.default_value
      if peak_indx == 0:
        peak_indx = self.default_value
        
      start = starts[-1] if len(starts)>0 else peak_indx-100
      stop = stops[0]+peak_indx if len(stops)>0 else peak_indx+100

        
      return histo, start, stop, peak_indx 
 
    def returnLane(self):
      return self.lane
                       
    def returnClip(self):
      return self.imclip
    
    def returnHistogram(self):
      return self.smoothed
    
    def setOffset(self):
      return self.offset

  

# Main Program to Run

In [ ]:
import glob
import numpy as np
import cv2
import pickle
from moviepy.editor import VideoFileClip


src_dir = './'
out_dir = './all_together_output/'


#comment and uncomment the line below to select different
#movies.  Be careful because file output sizes are large
#and can take 10 minutes to write

#movies = glob.glob(src_dir+'project_video.mp4')
#movies = glob.glob(src_dir+'challenge_video.mp4')
movies = glob.glob(src_dir+'harder_challenge_video.mp4')
#movies = glob.glob(src_dir+'*.mp4')

kernel = np.ones((2,2),np.uint8)

#make these line points holder global
#left_start_inds,left_stop_inds,left_peak_inds,left_mean_inds = [],[],[],[]
#right_start_inds,right_stop_inds,right_peak_inds,right_mean_inds = [],[],[],[]
l_inds = [[],[],[],[]]
r_inds = [[],[],[],[]]

#flags to toggle depeding if refiting of the polylines is needed
left_poly_update_status = False
right_poly_update_status = False

#used to count how many times polyline updating is skipped
left_no_pix = 0
right_no_pix = 0

#stores the present history of lines (about 20)
#this is used for averaging and smoothing out outlier 
left_xfits = [[],[],[],[]]
right_xfits = [[],[],[],[]]
center_fits = [[]]

#stores present history of curvatures and offsets
curvatures = []
offsets = []
#movies = glob.glob(src_dir+'harder_challenge_video.mp4')
#yp is global; it only needs to be calculated once
yp = np.linspace(1, 720, 100)
overall_mask = np.zeros([720,1280], np.uint8)

#ToDo: determine left and right side with histo
left = [0,640]
right = [640, 1280]

pts = np.array([[581,477],[699,477],[896,675],[384,675]],dtype = "float32")
dst = np.array([[384,0],[896,0],[896,720],[384,720]],dtype = "float32")
M = cv2.getPerspectiveTransform(pts, dst)
print(M)
Minv = cv2.invert(M)
print(Minv)

Mt = cv2.getPerspectiveTransform(dst, pts)
print(Mt)

##Loop over movies in the src directory
with open('camera.pickle', 'rb') as handle:
    (dist, mtx) = pickle.load(handle)

framenumber = 0

for movie in movies:
   print(movie)
   outfile = movie.replace(src_dir,out_dir)
   
   input_clip = VideoFileClip(movie)
   #start = input_clip.duration/2
   #input_clip = input_clip.subclip(1,10)
   
   #print(type(process_image))
   clip = input_clip.fl_image(process_image) 
   framenumber += 1
    
   clip.write_videofile(outfile, audio=False)

[[ -6.15990375e-01  -1.53219828e+00   1.03423384e+03]
 [ -5.32907052e-15  -2.23996500e+00   1.06846331e+03]
 [ -7.26415456e-18  -2.39405982e-03   1.00000000e+00]]
(1.0, array([[ -1.62340199e+00,   4.81823264e+00,  -3.46912750e+03],
       [  1.24175529e-14,   3.14465347e+00,  -3.35994684e+03],
       [  1.79357214e-17,   7.52848851e-03,  -7.04391371e+00]]))
[[  2.30468750e-01  -6.84027778e-01   4.92500000e+02]
 [ -5.32907052e-15  -4.46435547e-01   4.77000000e+02]
 [ -6.07153217e-18  -1.06879340e-03   1.00000000e+00]]
./harder_challenge_video.mp4
max at end of edge summation 2
max at end of edge summation 1
max at end of edge summation 1
threshold 2
percent activated pixels 0.01674045138888889
left overlapping pixels 0; percent of total 0.0
left_poly
right overlapping pixels 0; percent of total 0.0
right_poly
****update left mask run
****update right mask run
[-5.7708167925306768e-05, 0.19194461774832605, 190.1829376659951]
[MoviePy] >>>> Building video ./all_together_output/harder_chal


  0%|          | 0/1200 [00:00<?, ?it/s]

max at end of edge summation 2


max at end of edge summation 1
max at end of edge summation 1
threshold 2
percent activated pixels 0.01674045138888889
left overlapping pixels 13658; percent of total 0.8575913600401859
left_poly
right overlapping pixels 29570; percent of total 0.6718622193947106
right_poly



  0%|          | 1/1200 [00:00<10:38,  1.88it/s]

...


 33%|███▎      | 398/1200 [03:57<08:04,  1.66it/s]

max at end of edge summation 1
threshold 2
percent activated pixels 0.021223958333333334
left overlapping pixels 19457; percent of total 1.0
right overlapping pixels 20856; percent of total 0.9991855507114454
****update left mask run
****update right mask run
[0.00023981086553789041, -0.51902347231008816, 556.54503785637019]


max at end of edge summation 2
max at end of edge summation 2



 33%|███▎      | 399/1200 [03:57<08:05,  1.65it/s]

max at end of edge summation 1
threshold 2
percent activated pixels 0.022994791666666667
left overlapping pixels 16431; percent of total 1.0
right overlapping pixels 28356; percent of total 0.9980290018302126
****update left mask run
****update right mask run
[0.0018196113911507466, -1.7462605525057311, 724.79804085630906]


## 